In [3]:
PROJECT="vertex-ai-by-example"
COMPONENT_NAME="bash-io"
BUCKET_NAME=f"gs://{PROJECT}"
JOBSPEC_FILENAME = f"{COMPONENT_NAME}-jobspec.json"
PIPELINE_ROOT = f"{BUCKET_NAME}/{COMPONENT_NAME}"
print(BUCKET_NAME)

gs://vertex-ai-by-example


In [25]:
!gcloud config set project $PROJECT
!gcloud services enable notebooks.googleapis.com
!gcloud services enable cloudbuild.googleapis.com
!gcloud services enable aiplatform.googleapis.com
!echo "Note: it can take a while for service accounts to get created with correct permissions"


Updated property [core/project].
Operation "operations/acf.p2-662478959901-9f44e5a9-2d34-4266-bbd3-42574aaae7f6" finished successfully.
Note: it can take a while for service accounts to get created with correct permissions


In [17]:
!gsutil mb $BUCKET_NAME

Creating gs://vertex-ai-by-example/...


In [135]:
!gcloud builds submit --tag gcr.io/$PROJECT/$COMPONENT_NAME:latest .

Creating temporary tarball archive of 10 file(s) totalling 46.3 KiB before compression.
Uploading tarball of [.] to [gs://vertex-ai-by-example_cloudbuild/source/1637008627.842577-fa5964ef101f4fc7bf64b4b78fd92907.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/vertex-ai-by-example/locations/global/builds/8e5f3742-31e8-4c06-9045-c886ae58fba6].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/8e5f3742-31e8-4c06-9045-c886ae58fba6?project=662478959901].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "8e5f3742-31e8-4c06-9045-c886ae58fba6"

FETCHSOURCE
Fetching storage object: gs://vertex-ai-by-example_cloudbuild/source/1637008627.842577-fa5964ef101f4fc7bf64b4b78fd92907.tgz#1637008628220592
Copying gs://vertex-ai-by-example_cloudbuild/source/1637008627.842577-fa5964ef101f4fc7bf64b4b78fd92907.tgz#1637008628220592...
/ [1 files][  6.3 KiB/  6.3 KiB]                                                
Operation co

In [17]:
!pip install kfp --upgrade

In [136]:
from kfp.v2 import dsl, components, compiler
from kfp import components

bash_io_step = components.load_component_from_file("./component.yaml")

@dsl.pipeline(
    name="bash-io-pipeline", 
    pipeline_root=PIPELINE_ROOT
)
def bash_io_pipeline(project_id: str):
    bash_io_step( 
                input_1_parameter="This is input1", 
                input_2_parameter=42,
                # input_3_parameter_path="this is input2, perhaps quite large so will be delivered to this component via a path",
                # input_artifact_3_path="This could be a very long string or an artefact (e.g. dataset) and will be written to a file in gcs"
                ) \
            .set_cpu_limit("500m") \
            .set_memory_limit("500M") 

compiler.Compiler().compile(
    pipeline_func=bash_io_pipeline, 
    package_path=JOBSPEC_FILENAME
)
print(jobspec_filename)


bash-io-jobspec.json


In [139]:
import google.cloud.aiplatform as aip

job = aip.PipelineJob(
    display_name=f"{COMPONENT_NAME}-pipeline",
    template_path=jobspec_filename,
    enable_caching=False,
    # pipeline_root=pipeline_root_path,
    parameter_values={
        'project_id': 'p1'
    }
)


job.run()
print(job)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/662478959901/locations/us-central1/pipelineJobs/bash-io-pipeline-20211115213252
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/662478959901/locations/us-central1/pipelineJobs/bash-io-pipeline-20211115213252')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/bash-io-pipeline-20211115213252?project=662478959901
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/662478959901/locations/us-central1/pipelineJobs/bash-io-pipeline-20211115213252 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/662478959901/locations/us-central1/pipelineJobs/b